In [1]:
from models.model_factory import get_model, save_weights
from preprocessing.data import get_loaders, train_test_split, calculate_mean_std, create_datasets
from preprocessing.imbalanced_dataset import get_weighted_loss, get_loaders_with_oversampling
from training.training import train_model
from results.evaluation import evaluate_model
from results.visualizations import plot_confusion_matrix, plot_roc_curves, plot_precision_recall_curve
from utils.seed import set_seed
import torch

In [2]:
data_path = 'data'
batch_size = 32
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
set_seed(42)

mean, std = calculate_mean_std(data_path, batch_size)
print(f"Mean: {mean}, Std: {std}")

Mean: tensor([0.0985]), Std: tensor([0.1802])


In [4]:
train_loader, test_loader = get_loaders(data_path, mean, std, batch_size)

# CNN baseline

In [8]:
set_seed(42)

model = get_model('cnn_baseline', in_channels = 1, num_classes = 4)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

In [9]:
trained_model = train_model(model, train_loader, test_loader, criterion, optimizer, scheduler, device, num_epochs = 10)

Epoch 1/10 - Time: 228.96s 
    Train Loss: 0.8279 | Train Acc: 0.6399
    Test Loss: 0.7192 | Test Acc: 0.6834 | Test F1: 0.6928
Epoch 2/10 - Time: 227.50s 
    Train Loss: 0.6639 | Train Acc: 0.7159
    Test Loss: 0.7724 | Test Acc: 0.6751 | Test F1: 0.6709
Epoch 3/10 - Time: 226.18s 
    Train Loss: 0.5919 | Train Acc: 0.7506
    Test Loss: 0.6008 | Test Acc: 0.7540 | Test F1: 0.7606
Epoch 4/10 - Time: 225.46s 
    Train Loss: 0.5482 | Train Acc: 0.7715
    Test Loss: 0.5754 | Test Acc: 0.7658 | Test F1: 0.7663
Epoch 5/10 - Time: 224.81s 
    Train Loss: 0.5217 | Train Acc: 0.7895
    Test Loss: 0.5571 | Test Acc: 0.7728 | Test F1: 0.7788
Epoch 6/10 - Time: 224.61s 
    Train Loss: 0.4713 | Train Acc: 0.8034
    Test Loss: 0.5492 | Test Acc: 0.7736 | Test F1: 0.7804
Epoch 7/10 - Time: 224.47s 
    Train Loss: 0.4485 | Train Acc: 0.8190
    Test Loss: 0.5367 | Test Acc: 0.7802 | Test F1: 0.7872
Epoch 8/10 - Time: 225.35s 
    Train Loss: 0.4328 | Train Acc: 0.8243
    Test Loss: 0.51

In [10]:
save_weights(trained_model, 'cnn_baseline.pth')

# ResNet18 + weighted loss

In [5]:
model = get_model('resnet18', in_channels = 1, num_classes = 4)
criterion = get_weighted_loss(data_path)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

In [6]:
print(f"Loss weights: {criterion.weight}")

Loss weights: tensor([0.8614, 0.5180, 0.3055, 2.3151])


In [7]:
trained_model = train_model(model, train_loader, test_loader, criterion, optimizer, scheduler, device, num_epochs = 10)

Epoch 1/10 - Time: 2134.09s 
    Train Loss: 0.7135 | Train Acc: 0.6766
    Test Loss: 0.7612 | Test Acc: 0.6744 | Test F1: 0.6892
Epoch 2/10 - Time: 2146.03s 
    Train Loss: 0.5709 | Train Acc: 0.7442
    Test Loss: 0.6046 | Test Acc: 0.7858 | Test F1: 0.7739
Epoch 3/10 - Time: 2078.28s 
    Train Loss: 0.5076 | Train Acc: 0.7744
    Test Loss: 0.5026 | Test Acc: 0.8068 | Test F1: 0.8052
Epoch 4/10 - Time: 2077.34s 
    Train Loss: 0.4651 | Train Acc: 0.7942
    Test Loss: 1.3313 | Test Acc: 0.5417 | Test F1: 0.5408
Epoch 5/10 - Time: 2030.93s 
    Train Loss: 0.4291 | Train Acc: 0.8139
    Test Loss: 0.4834 | Test Acc: 0.8099 | Test F1: 0.8107
Epoch 6/10 - Time: 1550.61s 
    Train Loss: 0.3240 | Train Acc: 0.8575
    Test Loss: 0.3672 | Test Acc: 0.8612 | Test F1: 0.8620
Epoch 7/10 - Time: 2150.25s 
    Train Loss: 0.2847 | Train Acc: 0.8767
    Test Loss: 0.3406 | Test Acc: 0.8763 | Test F1: 0.8771
Epoch 8/10 - Time: 2206.56s 
    Train Loss: 0.2701 | Train Acc: 0.8837
    Test Lo

In [8]:
save_weights(trained_model, 'resnet18_weighted_loss.pth')

In [12]:
weighted_loss_results = evaluate_model(trained_model, test_loader, device, num_classes = 4)

In [13]:
set_seed(42)

# Resnet18

In [14]:
model = get_model('resnet18', in_channels = 1, num_classes = 4)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

In [15]:
trained_model = train_model(model, train_loader, test_loader, criterion, optimizer, scheduler, device, num_epochs = 10)

Epoch 1/10 - Time: 1398.66s 
    Train Loss: 0.6666 | Train Acc: 0.7372
    Test Loss: 0.9394 | Test Acc: 0.5901 | Test F1: 0.6169
Epoch 2/10 - Time: 1413.89s 
    Train Loss: 0.5411 | Train Acc: 0.7908
    Test Loss: 0.7856 | Test Acc: 0.6961 | Test F1: 0.7214
Epoch 3/10 - Time: 1477.33s 
    Train Loss: 0.4790 | Train Acc: 0.8161
    Test Loss: 0.5757 | Test Acc: 0.7693 | Test F1: 0.7512
Epoch 4/10 - Time: 1397.96s 
    Train Loss: 0.4382 | Train Acc: 0.8364
    Test Loss: 0.6449 | Test Acc: 0.7792 | Test F1: 0.7549
Epoch 5/10 - Time: 1846.21s 
    Train Loss: 0.4097 | Train Acc: 0.8484
    Test Loss: 0.4916 | Test Acc: 0.8106 | Test F1: 0.8148
Epoch 6/10 - Time: 1692.07s 
    Train Loss: 0.3117 | Train Acc: 0.8864
    Test Loss: 0.3160 | Test Acc: 0.8838 | Test F1: 0.8810
Epoch 7/10 - Time: 1637.24s 
    Train Loss: 0.2858 | Train Acc: 0.8972
    Test Loss: 0.2864 | Test Acc: 0.8940 | Test F1: 0.8937
Epoch 8/10 - Time: 1451.74s 
    Train Loss: 0.2819 | Train Acc: 0.8963
    Test Lo

In [16]:
save_weights(trained_model, 'resnet18_basic.pth')

In [17]:
results = evaluate_model(trained_model, test_loader, device, num_classes = 4)

In [4]:
train_loader, test_loader = get_loaders_with_oversampling(data_path, batch_size, mean, std)

# ResNet18 + oversampling

In [5]:
set_seed(42)

model = get_model('resnet18', in_channels = 1, num_classes = 4)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

In [6]:
trained_model = train_model(model, train_loader, test_loader, criterion, optimizer, scheduler, device, num_epochs = 10)

Epoch 1/10 - Time: 3403.72s 
    Train Loss: 0.6481 | Train Acc: 0.7314
    Test Loss: 0.8614 | Test Acc: 0.7247 | Test F1: 0.6848
Epoch 2/10 - Time: 2273.91s 
    Train Loss: 0.5045 | Train Acc: 0.8002
    Test Loss: 1.0346 | Test Acc: 0.6316 | Test F1: 0.6309
Epoch 3/10 - Time: 2427.61s 
    Train Loss: 0.4415 | Train Acc: 0.8245
    Test Loss: 0.6156 | Test Acc: 0.7721 | Test F1: 0.7763
Epoch 4/10 - Time: 2507.50s 
    Train Loss: 0.4083 | Train Acc: 0.8422
    Test Loss: 1.1597 | Test Acc: 0.5693 | Test F1: 0.5778
Epoch 5/10 - Time: 2214.17s 
    Train Loss: 0.4045 | Train Acc: 0.8440
    Test Loss: 0.5293 | Test Acc: 0.7776 | Test F1: 0.7799
Epoch 6/10 - Time: 2161.64s 
    Train Loss: 0.2823 | Train Acc: 0.8967
    Test Loss: 0.3205 | Test Acc: 0.8796 | Test F1: 0.8796
Epoch 7/10 - Time: 2114.55s 
    Train Loss: 0.2538 | Train Acc: 0.9048
    Test Loss: 0.3235 | Test Acc: 0.8817 | Test F1: 0.8825
Epoch 8/10 - Time: 2150.28s 
    Train Loss: 0.2441 | Train Acc: 0.9087
    Test Lo

In [7]:
save_weights(trained_model, 'resnet18_oversampling.pth')